In [197]:
from IPython.core.display import HTML
import jieba

class SearcherIIndex():
    """倒排索引文本搜索实现类
    
    用倒排索引
    利用Python的集合运算，来实现候选结果集之间交、并运算
    
    Attributes:
        index: 检索使用的倒排索引
        max_id: 当前索引的文档最大ID
        doc_list: 索引文档原文
    """
    def __init__(self, docs_file): 
        """初始化，用文件中的文本行构建倒排索引
        
        Args:
            docs_file:包含带索引文档(文本)的文件名
            
        """
        self.index = dict()    
        self.max_id = 0
        self.doc_list = [] 
        
        with open(docs_file, 'r') as f:
            docs_data = f.read()
        
        for doc in docs_data.split():
            self.add_doc(doc)

    def add_doc(self, doc):
        """向索引中添加新文档
        
        Args:
            doc:待检索的文档(文本)
        
        Returns:
            新增文档ID
        """
        self.doc_list.append(doc)
        for term in list(jieba.cut_for_search(doc)):
            #构建和更新各Term对应的Posting(集合)
            if term in self.index: 
                self.index[term].add(self.max_id)
            else:
                self.index[term] = set([self.max_id])
        self.max_id += 1
        return self.max_id - 1
    
    def word_match(self, word):
        """从倒排索引中获取包含word的候选文档ID集合
        
        Args:
            word:待检索的词(短语)
            
        Returns：
            包含待检索词(短语)的文档ID集合
        """
        result = None
        for term in list(jieba.cut(word)):
            if result is None:
                result = self.index.get(term, set())
            else:
                result = result & self.index.get(term, set())
        if result is None:
            result = set()
        return result

    def conv_query(self, query):
        """将用户的查询转换成用eval可运行、返回结果ID集合的代码段
        
        Args:
            query:待转换的原始查询字符串
        
        Returns:
            转换完成可通过eval执行返回ID集合的代码段字符串
        """
        query_new_parts = []
        all_parts = list(jieba.cut(query))
        idx = 0
        cache = '' #缓存变量，用于回收分词过程被切开的短语片段
        count_parts = len(all_parts)
        while idx < count_parts:
            if all_parts[idx] == '(' or all_parts[idx] == ')':
                query_new_parts.append(all_parts[idx])
            elif all_parts[idx] == ' ':
                query_new_parts.append(' ')
            elif all_parts[idx] in ('and', 'AND', '+'):
                query_new_parts.append('&')
            elif all_parts[idx] in ('or', 'OR'):
                query_new_parts.append('|')
            elif all_parts[idx] in ('not', 'NOT', '-'):
                query_new_parts.append('-')
            elif (idx + 1 < count_parts #被分词切开的短语部分回收至缓存
                  and all_parts[idx+1] not in (' ', ')')): 
                cache += all_parts[idx]
            elif (idx + 2 < count_parts #处理词间空格的形式
                  and all_parts[idx+1] == " " 
                  and all_parts[idx+2] not in ('(', ')', 'and', 'AND', '+', 'or', 'OR', 'NOT', 'not', '+', '-', ' ')): 
                query_new_parts.append("self.word_match('{}') & ".format(all_parts[idx]))
                idx += 2
                continue
            else:
                query_new_parts.append("self.word_match('{}')".format(cache + all_parts[idx]))
                cache = '' #合并完成清空缓存
            idx += 1
        query_new = ''.join(query_new_parts)
        return query_new

    def highlighter(self, doc, word):
        """用word对doc进行HTML高亮
        
        Args:
            doc:需要高亮的文档
            word:要进行高亮的关键词(查询)
            
        Returns:
            返回对关键词(查询)进行高亮的文档
        """
        for part in list(jieba.cut(word)):
            #TODO(CHG):短语高亮需要先分词
            if part not in ('(', ')', 'and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                doc = doc.replace(part, '<span style="color:red">{}</span>'.format(part))
        return doc

    def search(self, query):
        """用query进行查询返回结果文档列表
        
        Args:
            query:用户的(复合)布尔查询字符串
            
        Returns:
            复合查询要求的(高亮)文档结果列表
        """
        result = []
        query_new = self.conv_query(query)
        print(query_new)
        for did in eval(query_new):
            result.append(self.highlighter(self.doc_list[did], query))
        return result

In [219]:
import string
class SearcherIIndexVII(SearcherIIndex):
    #判断ch是否和英文字母，数字等属于同一类字符
    def noNeedToBeCut(self,ch):
        return ch in string.ascii_letters \
                    or ch.isdigit() \
                    or ch in ('-', ':', '.')
    
    #自定义字串解析方式，中文一类，英文数字'-',':','.'一类，' ','(',')'一类
    def preProcessStr(self,str):
        result = []
        state_last = ''
        cache = ''
        for c in str:
            state_c = self.noNeedToBeCut(c)
            if c in (' ','(',')'):
                if cache:
                    result.append(cache)
                result.append(c)
                cache = ''
                state_last = '' 
            else:
                if state_c == state_last:
                    cache += c
                else:
                    if state_last!='':
                        result.append(cache)
                    cache = c
                state_last = state_c
        if cache:
            result.append(cache)
        return result
    
    #解析文档，将非英文类的词语使用cut_for_search分开
    def parse_doc(self,doc):
        result = []
        for x in self.preProcessStr(doc):
            if x==' ':
                result.append(' ')
            else:
                if self.noNeedToBeCut(x[0]):
                    result.append(x)
                else:
                    result.extend(list(jieba.cut_for_search(x)))
        return result
    
    #建立倒排索引
    def add_doc(self, doc):
        self.doc_list.append(doc)
        for term in self.parse_doc(doc):
            #构建和更新各Term对应的Posting(集合)
            if term in self.index:
                self.index[term].add(self.max_id)
            else:
                self.index[term] = set([self.max_id])
        self.max_id += 1
        return self.max_id - 1
    
    def word_match(self, word,flagCut):
        result=None
        if flagCut:
            for term in list(jieba.cut(word)):
                if result==None:
                    result = self.index.get(term, set())
                else:
                    result=result&self.index.get(term, set())
        else:
            result = self.index.get(word, set())
        if result==None:
            result=set()
        return result
    
    #解析查询语句，如果有中文词语，则使用cut分词
    def conv_query(self, query):
        query_new_parts = []
        all_parts = self.preProcessStr(query)
        idx = 0
        while idx < len(all_parts):
            if all_parts[idx] == '(' or all_parts[idx] == ')':
                query_new_parts.append(all_parts[idx])
            elif all_parts[idx] == ' ':
                query_new_parts.append(' ')
            elif all_parts[idx] in ('and', 'AND','&'):
                query_new_parts.append('&')
            elif all_parts[idx] in ('or', 'OR','|'):
                query_new_parts.append('|')
            elif all_parts[idx] in ('not', 'NOT','-'):
                query_new_parts.append('-')
            else:
                if self.noNeedToBeCut(all_parts[idx][0]):
                    query_new_parts.append("self.word_match('{}',False)".format(all_parts[idx]))
                else:
                    query_new_parts.append("self.word_match('{}',True)".format(all_parts[idx]))
            idx += 1
            
        #判断query_new_parts中的' '是否需要替换成and，未解决连续空格问题
        for i in range(1,len(query_new_parts)-1):
            if query_new_parts[i]==' ':
                if not (query_new_parts[i-1] in ['(',')','&','|','-'] or 
                        query_new_parts[i+1] in ['(',')','&','|','-']):
                    query_new_parts[i]='&'
                    
        query_new = ''.join(query_new_parts)
        return query_new

In [220]:
searcher=SearcherIIndexVII('tiles.txt')

In [255]:
query='Mate30 or 3-0'

In [256]:
for doc in searcher.search(query):
    display(HTML(doc))

self.word_match('Mate30',False) | self.word_match('3-0',False)
